# Goals

1. Create Metrics DF with all errorterms from all models
    - Structure: Index = Modelname + Category, Columns = Error Metric + Horizon, Values=Metrics
    
    
2. Create Value DF with all forecasts from all models 
    - Structure: Index = Date, Columns = Modelname, Values=Forecasts 

## Setup

In [ ]:
# library
import pandas as pd
import numpy as np 

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.metrics import mean_absolute_percentage_error as mape
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import mean_absolute_error as mae

from itertools import combinations
# Get custom utils functions
import sys
sys.path.append('/Users/ludwigbaunach/Documents/Studium/PhD/Alaiko/Paper_1_Project/Main/src')

from utils.data_split import ml_data_date_split
from utils.model_utils import rmse as RMSE
from utils.evaluation_of_df_ts import create_error_dataframes

In [ ]:
#data 
L_3 = pd.read_pickle("../data/processed/L_3_test.pkl")

# time series
ts_results = pd.read_pickle("../data/modelling_results/ts_results_all.pickle")

lgbm_results = pd.read_pickle("../data/modelling_results/lgbm_results.pickle")
lgbm_results_s = pd.read_pickle("../data/modelling_results/lgbm_results_selected_f.pickle")
lgbm_params = pd.read_pickle("../data/modelling_results/lgbm_params.pickle")
lgbm_importance = pd.read_pickle("../data/modelling_results/lgbm_importance.pickle")

# xgb
xgb_results = pd.read_pickle("../data/modelling_results/xgb_results.pickle")
xgb_params = pd.read_pickle("../data/modelling_results/xgb_params.pickle")
xgb_importance = pd.read_pickle("../data/modelling_results/xgb_importance.pickle")

# LSTM
lstm_results = pd.read_pickle("../data/modelling_results/lstm_results.pickle")
# nhits

nhits_results = pd.read_pickle("../data/modelling_results/nhits_results.pickle")

## Explore Data

In [ ]:
lgbm_results.keys()
#lgbm_results

In [ ]:
ts_results.keys()
#ts_results

In [ ]:
lstm_results.keys()
lstm_results["L_6"]["backtest"]

In [ ]:
nhits_results.keys()
nhits_results["L_6"]["backtest"]

## Individual Metrics

### ts_results

In [ ]:
# example
ts_model_names = ts_results["L_3"]["pred"].columns[0:5]
ts_model_names

In [ ]:
def calculate_rmse_mape(true_values, predicted_values):
    
    rmse_score = np.sqrt(mse(true_values, predicted_values))
    mape_score = mape(true_values, predicted_values)
    
    return rmse_score, mape_score

In [ ]:
def create_error_df_ts(ts_results, lgbm_results):
    data = []
    index = []
    # Loop through categories
    for category_name, _ in ts_results.items():
        # Get model names
        ts_model_names = ts_results[category_name]["pred"].columns[0:5]
        
        for model_name in ts_model_names:
            # Access backtest and prediction data frame for the current level and category
            ts_backtest = ts_results[category_name]['backtest'].reset_index().groupby(["index"]).sum()
            ts_pred = ts_results[category_name]['pred'].reset_index().groupby(["index"]).sum()

            # Access actual values for backtest and prediction
            back_actual = lgbm_results["L_3_Time_Momentum_Lag"]["backtest"].groupby(["date"]).sum().actual
            pred_actual = lgbm_results["L_3_Time_Momentum_Lag"]["pred"].groupby(["date"]).sum().actual

            # Calculate RMSE and MAPE for backtest
            rmse_backtest, mape_backtest = calculate_rmse_mape(back_actual, ts_backtest[model_name])

            # Calculate RMSE and MAPE for forecast
            rmse_pred, mape_pred = calculate_rmse_mape(pred_actual, ts_pred[model_name])

            data.append([rmse_backtest, rmse_pred, mape_backtest, mape_pred])
            index.append(f"{category_name}_{model_name}")

    # Create column names
    columns = [f"val_rmse", f"pred_rmse", f"val_mape", f"pred_mape"]

    # Create the DataFrame
    df = pd.DataFrame(data, columns=columns, index=index)

    return df


In [ ]:
ts_error = create_error_df_ts(ts_results, lgbm_results)

In [ ]:
ts_error.sort_values("pred_rmse")

### lgbm_resuts

In [ ]:
# params
#params_df = pd.DataFrame.from_dict(lgb_params, orient="index")
# save as exel
#params_df.to_excel("../data/modelling_results/lgbm_params.xlsx")

In [ ]:
def create_error_df_tree(tree_results):
    # Initialize an empty list to store the results
    metrics_data = []

    # Iterate over the dataframes and categories

    for category_name, _ in tree_results.items():
        # Access backtest data frame for the current level and category
        backtest = tree_results[f"{category_name}"]['backtest'].groupby("date").sum()
        # Access prediction data frame for the current level and category
        pred = tree_results[f"{category_name}"]['pred'].groupby("date").sum()

        # Calculate RMSE
        val_rmse = np.sqrt(mse(backtest.actual, backtest.pred))
        pred_rmse = np.sqrt(mse(pred.actual, pred.pred))

        # Calculate MAPE
        val_mape = mape(backtest.actual, backtest.pred)
        pred_mape = mape(pred.actual, pred.pred)

        # Calculate MAE
        val_mae = mae(backtest.actual, backtest.pred)
        pred_mae = mae(pred.actual, pred.pred)

        # Append the results to the metrics_data list
        metrics_data.append({
            'category': category_name,
            'val_rmse': val_rmse,
            'pred_rmse': pred_rmse,
            'val_mape': val_mape,
            'pred_mape': pred_mape,
            #'val_mae': val_mae,
            #'pred_mae': pred_mae
        })
        
        lgbm_metrics = pd.DataFrame(metrics_data)
        
    return lgbm_metrics

In [ ]:
# lgbm errors
lgbm_error = create_error_df_tree(lgbm_results) #.sort_values("val_rmse")
lgbm_error.sort_values("pred_rmse")

In [ ]:
# xgb error
xgb_error = create_error_df_tree(xgb_results) 
xgb_error.sort_values("pred_rmse")

In [ ]:
# save to excel 
#df_metrics.to_excel("../data/modelling_results/lgbm_metrics.xlsx")

In [ ]:
#lstm_results["L_6"]["backtest"]

### Deep Learning Errors

In [ ]:
def create_error_df_dl(dl_results, lgbm_results, model_name):
    
    data = []
    index = []
    # Loop through categories
    for category_name, _ in dl_results.items():

        # Access backtest and prediction data frame for the current level and category
        dl_backtest = dl_results[category_name]['backtest'].groupby(["date"]).sum().pred
        dl_pred = dl_results[category_name]['pred'].groupby(["date"]).sum().pred

        # Access actual values for backtest and prediction
        back_actual = lgbm_results["L_3_Time_Momentum_Lag"]["backtest"].groupby(["date"]).sum().actual
        pred_actual = lgbm_results["L_3_Time_Momentum_Lag"]["pred"].groupby(["date"]).sum().actual
        
        # Calculate RMSE and MAPE for backtest
        rmse_backtest, mape_backtest = calculate_rmse_mape(back_actual, dl_backtest)

        # Calculate RMSE and MAPE for forecast
        rmse_pred, mape_pred = calculate_rmse_mape(pred_actual, dl_pred)

        data.append([rmse_backtest, rmse_pred, mape_backtest, mape_pred])
        index.append(f"{category_name}_{model_name}")

    # Create column names
    columns = [f"val_rmse", f"pred_rmse", f"val_mape", f"pred_mape"]

    # Create the DataFrame
    df = pd.DataFrame(data, columns=columns, index=index)

    return df

In [ ]:
lstm_error = create_error_df_dl(lstm_results, lgbm_results, "lstm")

In [ ]:
nhits_error = create_error_df_dl(nhits_results, lgbm_results, "nhits")

In [ ]:
nhits_error

## Ensemble  Metrics

Method:

- Select best model per level: L 6,5,4,3,2,
- select SARIMAX ARIMA and ES

- make combinations

### Custom functions

In [ ]:
def back_pred_df(ts_results, lgbm_results, xgb_results, lstm_results, nhits_results):
    
    # Lists to hold 'pred' series
    backtest_list = []
    pred_list = []
    
    # ts
    for key in ts_results.keys():
        for column in ts_model_names:
            backtest_series = ts_results[key]['backtest'].reset_index().groupby(["index"]).sum()[column]
            pred_series  = ts_results[key]['pred'].reset_index().groupby(["index"]).sum()[column]

            # Rename the series to the current key and column
            backtest_series.name = key + '_' + column
            pred_series.name = key + '_' + column

            # Add the series to the list
            backtest_list.append(backtest_series)
            pred_list.append(pred_series)

    # lgbm
    for key in lgbm_results.keys():

        backtest_series = lgbm_results[key]['backtest'].groupby(["date"]).sum().pred
        pred_series  = lgbm_results[key]["pred"].groupby(["date"]).sum().pred


        # Rename the series to the current key
        backtest_series.name = key + "_lgbm"
        pred_series.name = key + "_lgbm"

        # Add the series to the list
        backtest_list.append(backtest_series)
        pred_list.append(pred_series)

    # xgb
    for key in xgb_results.keys():

        backtest_series = xgb_results[key]['backtest'].groupby(["date"]).sum().pred
        pred_series  = xgb_results[key]["pred"].groupby(["date"]).sum().pred


        # Rename the series to the current key
        backtest_series.name = key + "_xgb"
        pred_series.name = key + "_xgb"

        # Add the series to the list
        backtest_list.append(backtest_series)
        pred_list.append(pred_series)

    # lstm 

    for key in lstm_results.keys():

        backtest_series = lstm_results[key]['backtest'].groupby(["date"]).sum().pred
        pred_series  = lstm_results[key]["pred"].groupby(["date"]).sum().pred


        # Rename the series to the current key
        backtest_series.name = key + "_lstm"
        pred_series.name = key + "_lstm"

        # Add the series to the list
        backtest_list.append(backtest_series)
        pred_list.append(pred_series)
        
    # nhits
    for key in lstm_results.keys():

        backtest_series = nhits_results[key]['backtest'].groupby(["date"]).sum().pred
        pred_series  = nhits_results[key]["pred"].groupby(["date"]).sum().pred


        # Rename the series to the current key
        backtest_series.name = key + "_nhits"
        pred_series.name = key + "_nhits"

        # Add the series to the list
        backtest_list.append(backtest_series)
        pred_list.append(pred_series)

        
    # Concatenate all series into one dataframe
    backtest_df = pd.concat(backtest_list, axis=1)
    pred_df = pd.concat(pred_list, axis=1)

    # attach actuals 
    # Access actual values for backtest and prediction
    backtest_df["actual"] = lgbm_results["L_3_Time_Momentum_Lag"]["backtest"].groupby(["date"]).sum().actual
    pred_df["actual"] = lgbm_results["L_3_Time_Momentum_Lag"]["pred"].groupby(["date"]).sum().actual
    
    # Set index names
    backtest_df.index.name = 'date'
    pred_df.index.name = 'date'
    
    return backtest_df, pred_df

In [ ]:
def create_ensemble(backtest_df, pred_df, list_of_selected_models):
    # Lists to hold 'pred' series
    backtest_list = []
    pred_list = []

    # create list of combinations
    model_pairs = list(combinations(list_of_selected_models, 2))

    for pair in model_pairs:
        # create ensembeles 
        backtest_ensemble = (backtest_df[pair[0]] + backtest_df[pair[1]]) / 2
        pred_ensemble = (pred_df[pair[0]] + pred_df[pair[1]]) / 2

        # Rename the series to the current key
        backtest_ensemble.name = pair[0] + "_" + pair[1]
        pred_ensemble.name = pair[0] + "_" + pair[1]

        # Add the series to the list
        backtest_list.append(backtest_ensemble)
        pred_list.append(pred_ensemble)

    back_ens_df = pd.concat(backtest_list, axis=1)
    pred_ens_df = pd.concat(pred_list, axis=1)
    
    # Set index names
    back_ens_df.index.name = 'date'
    pred_ens_df.index.name = 'date'

    return back_ens_df, pred_ens_df



In [ ]:
def ensemble_metrics(backtest_df, pred_df):
    
    # Initialize an empty list to store the results
    metrics_data = []

    # Iterate over the dataframes and categories

    for category_name in backtest_df.columns:
        # Access backtest data frame for the current level and category
        backtest = backtest_df[category_name]
        # Access prediction data frame for the current level and category
        pred = pred_df[category_name]

        # Calculate RMSE
        val_rmse = np.sqrt(mse(backtest_df.actual, backtest))
        pred_rmse = np.sqrt(mse(pred_df.actual, pred))

        # Calculate MAPE
        val_mape = mape(backtest_df.actual, backtest)
        pred_mape = mape(pred_df.actual, pred)

        # Calculate MAE
        val_mae = mae(backtest_df.actual, backtest)
        pred_mae = mae(pred_df.actual, pred)

        # Append the results to the metrics_data list
        metrics_data.append({
            'category': category_name,
            'val_rmse': val_rmse,
            'pred_rmse': pred_rmse,
            'val_mape': val_mape,
            'pred_mape': pred_mape,
            #'val_mae': val_mae,
            #'pred_mae': pred_mae
        })
        
        ensemble_metrics = pd.DataFrame(metrics_data)
        
    return ensemble_metrics

In [ ]:
def ensemble_stats(backtest_df, pred_df):
    # Initialize lists to store the values
    column_names = []
    means = []
    standard_deviations = []

    # Calculate mean and standard deviation for each column
    for i in backtest_df.columns:
        mean = backtest_df[i].mean()
        standard_deviation = backtest_df[i].std()

        # Append the results to the lists
        column_names.append(i)
        means.append(mean)
        standard_deviations.append(standard_deviation)

    # Create new DataFrame
    stats_df = pd.DataFrame({
        'Column Name': column_names,
        'Mean': means,
        'Standard Deviation': standard_deviations
    })
    
    return stats_df

In [ ]:
# Run Functions and get individual metrics (without ensembles)
back_df, pred_df = back_pred_df(ts_results, lgbm_results, xgb_results, lstm_results, nhits_results)
ind_metrics_df = ensemble_metrics(back_df, pred_df).sort_values("val_mape")

In [ ]:
# Get nest Models to create ensembles (to limit compexity)
list_of_selected_models = list(ind_metrics_df[1:11].category)
list_of_selected_models.append("L_4_sarimax")

# Create ensemble and save as df
back_ens_df, pred_ens_df = create_ensemble(back_df, pred_df, list_of_selected_models)

# Backtest and prediction values 
full_back_df = back_df.merge(back_ens_df,  left_index=True, right_index=True, how="left")
full_pred_df = pred_df.merge(pred_ens_df,  left_index=True, right_index=True, how="left")

# esemble metrics
ensemble_metrics_df = ensemble_metrics(full_back_df, full_pred_df).sort_values("pred_rmse")
# test
#ensemble_metrics_df.head(50)

In [ ]:
ensemble_metrics_df.head(20)

In [ ]:
lgbm_results["L_4_Time_Momentum_Lag"]["backtest"].groupby("date").sum().actual.plot()
lgbm_results["L_4_Time_Momentum_Lag"]["backtest"].groupby("date").sum().pred.plot()
ts_results["L_4"]["backtest"].reset_index().groupby("index").sum().sarimax.plot()
#full_back_df.L_4_Time_Momentum_Lag_lgbm_L_4_sarimax.plot()

In [ ]:
list_of_selected_models.append("L_4_sarimax")

In [ ]:
len(full_back_df.columns)

## SAVE

In [ ]:
import pickle

#with open('../data/modelling_results/ens_back_results.pickle', 'wb') as handle:
#    pickle.dump(full_back_df, handle, protocol=pickle.HIGHEST_PROTOCOL)

#with open('../data/modelling_results/ens_pred_results.pickle', 'wb') as handle:
#    pickle.dump(full_pred_df, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
# save to excel 
#ensemble_metrics_df.to_excel("../data/modelling_results/ensemble_metrics.xlsx")
#ind_metrics_df.to_excel("../data/modelling_results/ind_metrics.xlsx")

In [ ]:
len(pd.read_pickle("../data/modelling_results/ens_back_results.pickle").columns)

In [ ]:
len(pd.read_pickle("../data/modelling_results/ens_back_results_v2.pickle").columns)

# Significance

In [ ]:
from scipy.stats import ttest_rel, shapiro
from scipy.stats import wilcoxon

# Assuming your DataFrame is called 'df' and has columns 'actual', 'backtest', 'prediction'
df = pd.DataFrame(metrics_data)

test = df[df["category"] == "L_5_Time_Momentum_Lag"]

stat_backtest, p_backtest = shapiro(df['val_mae'])
stat_prediction, p_prediction = shapiro(df['pred_mae'])

alpha = 0.05  # Choose your significance level

if p_backtest < alpha or p_prediction < alpha:
    print("At least one of the MAE distributions is not normal. Consider using a non-parametric test.")
else:
    # Perform paired t-test
    t_stat, p_value = ttest_rel(df['mae_backtest'], df['mae_prediction'])

    if p_value < alpha:
        print("There is a significant difference between the two models (p-value: {:.5f})".format(p_value))
    else:
        print("There is no significant difference between the two models (p-value: {:.5f})".format(p_value))



In [ ]:

alpha1 = 0.05  # Set a significance level (e.g., 0.05)
alpha2 = 0.01  # Set a significance level (e.g., 0.01)

significance_results = []

for category_name, _ in lgb_params.items():
    # Access backtest data frame for the current level and category
    backtest_lgb = lgbm_tr[f"{category_name}"]['backtest'].groupby("date").sum()
    backtest_arima = ts_backtest.ARIMA

    # Access prediction data frame for the current level and category
    pred_lgb = lgbm_tr[f"{category_name}"]['pred'].groupby("date").sum()
    pred_arima = ts_forecast.ARIMA

    # Perform the Wilcoxon Signed-Rank Test for backtest errors
    stat_backtest, p_value_backtest = wilcoxon(backtest_lgb.actual - backtest_lgb.pred, backtest_lgb.actual - backtest_arima)

    # Perform the Wilcoxon Signed-Rank Test for prediction errors
    stat_pred, p_value_pred = wilcoxon(pred_lgb.actual - pred_lgb.pred, backtest_lgb.actual - pred_arima)

    significance_backtest = 0
    if p_value_backtest < alpha1:
        significance_backtest = 1
    if p_value_backtest < alpha2:
        significance_backtest = 2

    significance_pred = 0
    if p_value_pred < alpha1:
        significance_pred = 1
    if p_value_pred < alpha2:
        significance_pred = 2

    significance_results.append([category_name, significance_backtest, significance_pred])

# Create a DataFrame with the results
significance_df = pd.DataFrame(significance_results, columns=['Category', 'Backtest Significance', 'Prediction Significance'])

# save to excel 
#significance_df.to_excel("../data/modelling_results/lgbm_significance.xlsx")


# Feature Importance

In [ ]:
pd.read_pickle("../data/processed/L_4_test.pkl").columns

In [ ]:
# test drop
['tm_y_1',
 'tm_wm_cos',
 'event_Thanksgiving',
 'holiday_Zweiter_Weihnachtstag',
 'holiday_Ostermontag',
 'holiday_Neujahr',
 'holiday_Reformationstag',
 'holiday_Karfreitag',
 'event_Valentines_Day',
 'holiday_Pfingstmontag',
 'holiday_Erster_Mai',
 'holiday_Erster_Weihnachtstag',
 'holiday_Tag_der_Deutschen_Einheit',
 'holiday_Christi_Himmelfahrt',
 'tm_y_2']

In [ ]:
nr_freatures = 10
# Permutation Iportance of lgbm moddels
fig, axes = plt.subplots(6, 2, figsize=(12, 20))  # figsize in inches, changed to fit A4 dimensions
axes = axes.ravel()  # Flatten the axes array for easier indexing

# Iterate through the feature_importance dictionary items and get the level and value
for idx, (level, value) in enumerate(lgbm_importance.items()):
    df = value["df"].sort_index()
    box_array = value["box_array"]
    columns = df.feature_names
    sns.boxplot(x="importance", y="feature_names", data=pd.DataFrame(box_array, columns=columns)\
                .melt(var_name='feature_names', value_name='importance').sort_values("importance", ascending=False),#.head(nr_freatures), 
                ax=axes[idx], 
                order=df.sort_values("importances_mean", ascending=False).head(nr_freatures).feature_names,
                color="grey")
    axes[idx].set_title(f'{level}', fontsize=10)  # adjust fontsize
    axes[idx].set_xlabel('Importance', fontsize=8)  # adjust fontsize
    axes[idx].set_ylabel('Features', fontsize=8)  # adjust fontsize
    plt.yticks(rotation=0)

# Remove the unused subplot in case you have an odd number of plots
if len(lgbm_importance) % 2 == 1:
    fig.delaxes(axes[-1])

plt.tight_layout()

# Save the figure as a PNG file
##plt.savefig('../data/figures/all_feature_importance_per_cat_lgbm.png', dpi=300, bbox_inches='tight')

plt.show()

In [ ]:
#I made these calc:
df['tm_dy'] = df.date.dt.dayofyear.astype(np.int8)  # Day of year +
df['tm_dm'] = df.date.dt.day.astype(np.int8)  # Day of month +
df['tm_wy'] = df.date.dt.isocalendar().week.astype(np.int8)  # week of year +
df['tm_my'] = df.date.dt.month.astype(np.int8)  # month of year +
df['tm_y'] = df.date.dt.year  # year
df['tm_y'] = (df['tm_y'] - df['tm_y'].min()).astype(np.int8)  # year - min year = number of year
df['tm_wm'] = df['tm_dm'].apply(lambda x: math.ceil(x / 7)).astype(np.int8)  # number of week in month
df['tm_dw'] = df.date.dt.dayofweek.astype(np.int8)  # number of day in week # +
df['tm_w_end'] = (df['tm_dw'] >= 5).astype(np.int8)  # indicate Weekend

# Momentum Features: Rolling averages and standard dev 9 & 14 days
product_sales['q_roll_mean_9d'] = product_sales.groupby([i])['quantity'].shift(9).rolling(9, min_periods=1)\
    .mean().reset_index().quantity.fillna(0)
product_sales['q_roll_std_9d'] = product_sales.groupby([i])['quantity'].shift(9).rolling(9, min_periods=1)\
    .std().reset_index().quantity.fillna(0)
product_sales['q_roll_mean_14d'] = product_sales.groupby([i])['quantity'].shift(9).rolling(14, min_periods=1)\
    .mean().reset_index().quantity.fillna(0)
product_sales['q_roll_std_14d'] = product_sales.groupby([i])['quantity'].shift(9).rolling(14, min_periods=1)\
    .std().reset_index().quantity.fillna(0)

# Lag Features 9 days, 14 day, 28 days and 365 days
product_sales['q_lag_9d'] = product_sales.groupby([i])['quantity'].shift(periods=9).fillna(0)
product_sales['q_lag_14d'] = product_sales.groupby([i])['quantity'].shift(periods=14).fillna(0)
product_sales['q_lag_28d'] = product_sales.groupby([i])['quantity'].shift(periods=28).fillna(0)
product_sales['q_lag_365d'] = product_sales.groupby([i])['quantity'].shift(periods=365).fillna(0)

# average of 9, 14 and 28 days
product_sales["q_mean_lag_9_14_28"] = (product_sales['q_lag_9d'] + product_sales['q_lag_14d']
                                       + product_sales['q_lag_28d'])/3

# help me rename these featues to consise intuitive names
df.feature_names.to_list()

In [ ]:
import pandas as pd
import numpy as np
import math

value = lgbm_importance["L_6_Time_Momentum_Lag_Weather_Holiday"]

df = value["df"].sort_index()

# Assuming df is your DataFrame
# Rename the values in column feature_names not columns
rename_dict = {
    'tm_dm_cos': 'day_month_cos',
    'tm_dm_sin': 'day_month_sin',
    'tm_y_2': 'year_2',
    'tm_dy_sin': 'day_year_sin',
    'tm_w_end': 'is_weekend',
    'tm_dy_cos': 'day_year_cos',
    'tm_y_1': 'year_1',
    'tm_wy_sin': 'week_year_sin',
    'tm_y_0': 'year_0',
    'tm_wm_sin': 'week_month_sin',
    'tm_my_cos': 'month_year_cos',
    'tm_my_sin': 'month_year_sin',
    'tm_dw_sin': 'day_week_sin',
    'tm_dw_cos': 'day_week_cos',
    'tm_wm_cos': 'week_month_cos',
    'tm_wy_cos': 'week_year_cos',
    'q_roll_std_9d': 'quantity_rolling_std_9d',
    'q_lag_14d': 'quantity_lag_14d',
    'q_lag_28d': 'quantity_lag_28d',
    'q_mean_lag_9_14_28': 'quantity_mean_lag_9_14_28',
    'q_lag_9d': 'quantity_lag_9d',
    'q_roll_std_14d': 'quantity_rolling_std_14d',
    'q_lag_365d': 'quantity_lag_365d',
    'q_roll_mean_9d': 'quantity_rolling_mean_9d',
    'q_roll_mean_14d': 'quantity_rolling_mean_14d',
}

# Your DataFrame df now has renamed columns
df['feature_names'] = df['feature_names'].replace(rename_dict)

In [ ]:
box_array = value["box_array"]
columns = df.feature_names

fig, axes = plt.subplots(figsize=(15, 8))  # figsize in inches, changed to fit A4 dimensions

sns.boxplot(x="importance", y="feature_names",
            data=pd.DataFrame(box_array, columns=columns).melt(var_name='feature_names', 
                                                               value_name='importance').sort_values("importance", 
                                                                                                    ascending=False),
            ax=axes,
            order=df.sort_values("importances_mean", ascending=False).head(30).feature_names,
            color="grey")

#axes.set_title('Feature Importances: L6_Weather_Holiday_lgbm', fontsize=10)  # adjust fontsize
axes.set_xlabel('Importance', fontsize=10)  # adjust fontsize
axes.set_ylabel('Features', fontsize=10)  # adjust fontsize
plt.yticks(rotation=0)

plt.savefig('../data/figures/feature_importance_L6_all_features_lgbm.png', dpi=300, bbox_inches='tight')

plt.show()

# Visualisation

Best Performing from every level

In [ ]:
merged_df = back_df.append(pred_df)
merged_df

In [ ]:
best_ts_models = pd.Series(ts_error.sort_values("pred_rmse")[0:1].index.unique())
best_lgbm_models = pd.Series(lgbm_error.sort_values("pred_rmse")[0:1].category.unique())
#best_xgb_models = pd.Series(xgb_error[0:1].category.unique())
#best_lstm_models = pd.Series(lstm_error[0:1].index.unique())
best_nhits_models = pd.Series(nhits_error.sort_values("pred_rmse")[0:1].index.unique())
#best_ens_models = pd.Series(ensemble_metrics_df[2:3].category.unique())  # best 2!

# add "_lgbm" and "_xgb" to best_lgbm_models and best_xgb_models
best_lgbm_models = best_lgbm_models.map(lambda x: f'{x}_lgbm')
#best_xgb_models = best_xgb_models.map(lambda x: f'{x}_xgb')

list_of_best_models = pd.concat([best_ts_models, best_lgbm_models, best_nhits_models])

# Set the style for the plot
sns.set(style="whitegrid", font_scale=1)

# Create the time series plot
plt.figure(figsize=(15, 5))

# Plot each model
for model in list_of_best_models:
    plt.plot(merged_df.index, merged_df[model], label=model, linestyle="--", alpha=0.9)

# The rest of your code...
# Plot the Actuals with a thicker, solid line and a distinct color
plt.plot(merged_df.index, merged_df["actual"], label="Actuals", color="black", linewidth=1.5)

# Get the number of y-ticks
n = len(plt.gca().get_yticks())

# Set new y-tick labels
plt.yticks(np.linspace(min(pred_df["actual"]), max(pred_df["actual"]), n), range(1, n+1))

# Customize the plot
plt.xlabel("Date")
plt.ylabel("Scaled Daily Orders")
plt.legend(loc="best", fontsize=10)
# Hide y-axis values
#plt.yticks([])

plt.tight_layout()
# save
#plt.savefig('../data/figures/forecast_comparison_best_category.png', dpi=300, bbox_inches='tight')
# Show the plot
plt.show()

In [ ]:
# Set the style for the plot
sns.set(style="whitegrid", font_scale=1)

# Create the time series plot
plt.figure(figsize=(15, 5))

# Plot each model
for model in list_of_best_models:
    
    # add if statement. If model in pred_ens_df plot else plot with pred_df
    if model in pred_ens_df.columns:
        plt.plot(back_ens_df.index, back_ens_df[model], label=model, linestyle="--", alpha=0.9)
        
    else:
        plt.plot(back_df.index, back_df[model], label=model, linestyle="--", alpha=0.9)

# The rest of your code...
# Plot the Actuals with a thicker, solid line and a distinct color
plt.plot(back_df.index, back_df["actual"], label="Actuals", color="black", linewidth=1.5)

# Get the number of y-ticks
n = len(plt.gca().get_yticks())

# Set new y-tick labels
plt.yticks(np.linspace(min(pred_df["actual"]), max(pred_df["actual"]), n), range(1, n+1))

# Customize the plot
plt.xlabel("Date")
plt.ylabel("Scaled Daily Orders")
plt.legend(loc="best", fontsize=10)

# Hide y-axis values
#plt.yticks([])

plt.tight_layout()
# save
plt.savefig('../data/figures/backtest_comparison_best_category.png', dpi=300, bbox_inches='tight')
# Show the plot
plt.show()